In [1]:
from modeling import *
from data_import import *
from data_preparation import *

from tensorflow import keras

import warnings
warnings.filterwarnings("ignore")

2023-03-02 14:39:06.099043: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-02 14:39:07.686449: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-02 14:39:07.687170: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-02 14:39:07.687200: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

Set all input 

In [2]:
global db_config
db_config = {                       #Start server first
    'host': 'localhost',
    'database': 'python_db',
    'user': 'postgres',
    'password': 'postgres'
}

train_table_name = 'train_Bearing_Nasa'
test_table_name = 'test_Bearing_Nasa'
dow, month = None, None                                                             #! Get from UI
cat = None                                                                          #! Get from UI
mult_var = None                                                                     #! Get from UI
l_periods = None                                                                    #! Get from UI

In [3]:
def train_pipeline(train_table_name, db_config, dow=None, month=None, cat=None, mult_var=None, l_periods=None):
  df = read_table_from_postgres(train_table_name, db_config)
  df = data_prep(df, dow=dow, month=month, cat=cat, mult_var=mult_var, l_periods=l_periods)
  if_model = fit_IF(df)
  df_cleaned = predict_IF(if_model, df)
  fit_scaler(df_cleaned, train_table_name)
  df_scaled = apply_scaler(df_cleaned, train_table_name)

  #set_global(df_cleaned.shape[1])
  ae_fit(df_scaled, train_table_name)

In [4]:
df = train_pipeline(train_table_name, db_config)

Removing Time periods...
(736, 4)
Removing date
(736, 4)
Fixing Typos...
(736, 4)
Encoding categorical varible(s)...
(736, 4)
Bearing 1    float64
Bearing 2    float64
Bearing 3    float64
Bearing 4    float64
dtype: object
Process Nan...
Bearing 1 - 0%
Bearing 2 - 0%
Bearing 3 - 0%
Bearing 4 - 0%
(736, 4)
End of preprocessing

0.1854   -     0.7256   -     0.4772   -     0.7302   -     0.115   -     0.3626   -     0.2111   -     0.7399   -     0.4772   -     0.7305   -     0.1691   -     0.3893   -     0.2086   -     0.6876   -     0.4772   -     0.6774   -     0.1641   -     0.4135   -     0.264   -     0.7853   -     0.4772   -     0.7764   -     0.0881   -     0.338   -     0.2193   -     0.728   -     0.4772   -     0.7193   -     0.1475   -     0.3785   -     0.1827   -     0.6982   -     0.4772   -     0.7043   -     0.1563   -     0.3883   -     0.1943   -     0.7101   -     0.4772   -     0.7017   -     0.1436   -     0.4163   -     0.2581   -     0.7999   -     0.4481   -    

In [5]:
def predict_pipeline(train_table_name, test_table_name):
    df_train = read_table_from_postgres(train_table_name, db_config)
    df_test = read_table_from_postgres(test_table_name, db_config)
    df_train = df_train.drop_duplicates(keep='last')
    df_test = df_test.drop_duplicates(keep='last')
    df_train = df_train.reset_index(drop=True)
    df_test = df_test.reset_index(drop=True)

    df_train_scaled = apply_scaler(df_train, train_table_name)
    df_test_scaled = apply_scaler(df_test, train_table_name)

    ae_path = "../models/ae_" + str(train_table_name) + ".h5"
    model = keras.models.load_model(ae_path)

    result = cluster_data(model, df_train_scaled, df_test_scaled, df_test, test_table_name)
    write_dataframe_to_postgres(result, str(test_table_name) + "_Result", db_config)

In [7]:
predict_pipeline(train_table_name, test_table_name)

8/8 [==============================] - 0s 1ms/step

Global Reconstruction error threshold:  0.2898632982504219
Successfully wrote 246 rows to table test_Bearing_Nasa_Result in database python_db!
